# Prepate Data Folder Structure for YOLO format

The expected folder structure to train a Yolo Model is:

In [ ]:
YOLO_Project/
│
├── data/                     # Dataset and configuration files
│   ├── images/               # All images (train/validation)
│   │   ├── train/            # Training images
│   │   │   ├── img1.jpg
│   │   │   ├── img2.jpg
│   │   │   └── ...
│   │   └── val/              # Validation images
│   │       ├── img1.jpg
│   │       └── ...
│   │
│   └── labels/               # YOLO labels (bounding boxes)
│       ├── train/            # Training labels (each .txt corresponds to an image)
│       │   ├── img1.txt
│       │   └── ...
│       └── val/              # Validation labels
│           ├── img1.txt
│           └── ...
│
├── yolov5/                   # YOLOv5 repository (if using YOLOv5)
│   └── ...                   # YOLOv5 scripts, config files, etc.
│
├── yolov8/                   # YOLOv8 repository (if using YOLOv8)
│   └── ...                   # YOLOv8 scripts, config files, etc.
│
└── data.yaml                 # YOLO data configuration file (specifies paths, classes, etc.)


In [3]:
import os
import json
import random
import shutil
from sklearn.model_selection import train_test_split

# Paths to your data and annotations
images_dir = "/datax/Maack/ism_2024_2025/sample_solution/phase_2a/images"
annotations_path = "/datax/Maack/ism_2024_2025/sample_solution/phase_2a/annotations_0_index.json"
output_dir = "/datax/Maack/ism_2024_2025/sample_solution/phase_2a/yolo_training"

images_output_train = os.path.join(output_dir, "images/train")
images_output_val = os.path.join(output_dir, "images/val")
labels_output_train = os.path.join(output_dir, "labels/train")
labels_output_val = os.path.join(output_dir, "labels/val")

# Create necessary directories
os.makedirs(images_output_train, exist_ok=True)
os.makedirs(images_output_val, exist_ok=True)
os.makedirs(labels_output_train, exist_ok=True)
os.makedirs(labels_output_val, exist_ok=True)

# Load COCO annotations
with open(annotations_path) as f:
    coco_data = json.load(f)

# Function to convert COCO bbox to YOLO format
def coco_to_yolo(bbox, img_width, img_height):
    x_min, y_min, width, height = bbox
    x_center = (x_min + width / 2) / img_width
    y_center = (y_min + height / 2) / img_height
    width /= img_width
    height /= img_height
    return x_center, y_center, width, height

# Split images into train and val
image_files = [img['file_name'] for img in coco_data['images']]
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Create dictionaries to quickly access image information
image_dict = {img['id']: img for img in coco_data['images']}
annotations_dict = {}
for annotation in coco_data['annotations']:
    image_id = annotation['image_id']
    if image_id not in annotations_dict:
        annotations_dict[image_id] = []
    annotations_dict[image_id].append(annotation)

# Process images and annotations
for img_file in coco_data['images']:
    img_id = img_file['id']
    img_name = img_file['file_name']
    img_width = img_file['width']
    img_height = img_file['height']

    # Determine if the image is in train or val split
    if img_name in train_files:
        img_dest = os.path.join(images_output_train, os.path.basename(img_name))
        label_dest = os.path.join(labels_output_train, os.path.basename(img_name).replace(".jpg", ".txt"))
    else:
        img_dest = os.path.join(images_output_val, os.path.basename(img_name))
        label_dest = os.path.join(labels_output_val, os.path.basename(img_name).replace(".jpg", ".txt"))

    # Copy the image to the corresponding directory
    shutil.copy(os.path.join(images_dir, img_name), img_dest)

    # Write YOLO annotation file
    if img_id in annotations_dict:
        with open(label_dest, 'w') as label_file:
            for annotation in annotations_dict[img_id]:
                category_id = annotation['category_id']
                bbox = annotation['bbox']
                x_center, y_center, width, height = coco_to_yolo(bbox, img_width, img_height)
                label_file.write(f"{category_id} {x_center} {y_center} {width} {height}\n")

print("Dataset preparation complete!")

Dataset preparation complete!


# Create the needed data.yaml file

In [4]:
import yaml

# Create the needed data.yaml file

OUTPUT_YAML_DIR = "/datax/Maack/ism_2024_2025/sample_solution/phase_2a/yolo_training"

data_yaml = {
    "train": images_output_train,
    "val": images_output_val,
    "nc": len(coco_data['categories']),
    "names": [cat['name'] for cat in coco_data['categories']]
}

# Save the data.yaml file
with open(os.path.join(OUTPUT_YAML_DIR, "data.yaml"), 'w') as f:
    yaml.dump(data_yaml, f)

# Clone the Yolov5 Repo

### Clone the repo into the /yolo_training folder via

In [ ]:
cd /datax/Maack/ism_2024_2025/sample_solution/phase_2a/yolo_training # Adapt to your directory structre
git clone https://github.com/ultralytics/yolov5.git
cd yolov5
pip install -r requirements.txt

### Start Training

In [ ]:
cd into the yolov5 directory and run the following command to train the model on the custom dataset:

# Adapt the /path/to/data.yaml to the path of your data.yaml file
python train.py --img 640 --batch 16 --epochs 50 --data /path/to/data.yaml --cfg models/yolov5m.yaml --weights yolov5m.pt --name instrument_detection

# Prepare your Submission with script.py

1. Copy the yolov5 folder under yolo_training/yolov5 into the /submission/ folder
2. Delete the __pycache__/ folder and/or other unnecessary folders to have <250 overall files in the yolov5 folder (>250 files per folder makes uploading to your huggingFace model repo for submission difficult)
3. Now you can upload the content of submission/ folder to your HuggingFace Model Repo for submission